In [1]:
import pandas as pd
import numpy as np
import pickle


In [5]:
#constants
Data_path='C:\\Users\\gt\\End to End Data Science Project\\survey_results_public.csv'
Export_path='C:\\Users\\gt\\End to End Data Science Project\\preprocessed_data.pkl'

Replace_dict={
    'YearsCodePro':{'More than 50 years':51,'Less than 1 year':0},
    'YearsCode':{'More than 50 years':51,'Less than 1 year':0}
}

<br>

<br>

## Functions

In [21]:
def split_answers(data_series, delimiter=";"):
    """
    split multiple answers in a single string to a list of single strings
    representing a single answer
    
    Parameters:
    * data_series (pd.Series): String series with answers
    * delimiter (string): Another decimal integer 
                          Defaults to ";"
    
    Returns: (pd.Series) if column contains
    """
    #sub functions
    def is_splittable(pd_series, delimiter):
        """check if results multiple should be splitted - returns boolean"""
        return pd_series.str.contains(delimiter)
    
    def split_answer(pd_series,delimiter):
        """function to split single answer"""
        return pd_series.str.split(delimiter)
    # ------------------------------------
    
    #check if multiple answer exist - if non return original
    splittable_values=is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series
    #Else, split each value to a list
    modeified_series=split_answer(data_series, delimiter)
    
    #Replace Nans with empty lists
    null_values= modeified_series.isnull()
    modeified_series.loc[null_values]=modeified_series.loc[null_values].apply(lambda x:[])
    
    return modeified_series

<br>

<br>

## Processing

### Preprocess Data

In [10]:
raw_df = pd.read_csv(Data_path)
df = raw_df.copy()

**1. Replace Values and parse**

In [15]:
for col, replacement in Replace_dict.items():
    df[col]=df[col].replace(replacement).astype(np.float32)

**2. Split Multiple answers**

In [22]:
object_cols =df.select_dtypes(include="object").columns.tolist()
for col in object_cols:
    df[col]=split_answers(df[col])

<br>

<br>

### Visually verify results

In [27]:
i =df.sample(1).index[0]
print(raw_df['LanguageHaveWorkedWith'].iloc[i])
print(df['LanguageHaveWorkedWith'].iloc[i])

C;C++;Python
['C', 'C++', 'Python']


In [28]:
i =df.sample(1).index[0]
print(raw_df['DevType'].iloc[i])
print(df['DevType'].iloc[i])

Developer, front-end;Developer, full-stack;Developer, back-end;Designer;Engineering manager
['Developer, front-end', 'Developer, full-stack', 'Developer, back-end', 'Designer', 'Engineering manager']


In [30]:
i =df.sample(1).index[0]
print(raw_df['YearsCodePro'].iloc[i])
print(df['YearsCodePro'].iloc[i])

3
3.0


<br>

<br>

## Export Data

In [31]:
df.to_pickle(Export_path)